<a href="https://colab.research.google.com/github/Zaidan1140/Data-Cleansing-Movie-Data/blob/main/2318078DataCleansing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Langkah 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. Baca Dataset pelanggan yang kotor

In [6]:
# Load dataset
file_path = "/content/drive/MyDrive/Colab Notebooks/movies_data.csv"# ganti sesuai path di Colab
try:
    df_dirty = pd.read_csv(file_path, encoding='latin-1')
    print("Dataset loaded successfully with latin-1 encoding.")
except UnicodeDecodeError:
    print("UnicodeDecodeError: Could not decode the file with latin-1 encoding. Please try another encoding like 'cp1252' or 'ISO-8859-1'.")
    # You can try another encoding here if latin-1 fails
    # try:
    #     df_dirty = pd.read_csv(file_path, encoding='cp1252')
    #     print("Dataset loaded successfully with cp1252 encoding.")
    # except UnicodeDecodeError:
    #     print("UnicodeDecodeError: Could not decode the file with cp1252 encoding. Please check the file encoding.")
except FileNotFoundError:
    print(f"FileNotFoundError: The file was not found at {file_path}. Please check the file path.")

Dataset loaded successfully with latin-1 encoding.


In [7]:
# Use the already loaded dataframe
df = df_dirty

# Tampilkan semua baris data
print("=== Dataset Asli ===")
print(df.head())

print("\nInfo Dataset:")
print(df.info())

print("\nJumlah Data & Kolom:", df.shape)

=== Dataset Asli ===
              Movie        Director  Running time          Actor 1  \
0          13 Hours     Michael Bay           144    Toby Stephens   
1         16 Blocks  Richard Donner           102     Bruce Willis   
2          17 Again     Burr Steers           102    Matthew Perry   
3              1982    Tommy Oliver            90  Bokeem Woodbine   
4  2 Fast 2 Furious  John Singleton           107      Paul Walker   

            Actor 2          Actor 3   Genre    Budget  Box Office  \
0  James Badge Dale  David Costabile  Action  50000000    69400000   
1       David Zayas       Sasha Roiz  Action  52000000    65000000   
2    Hunter Parrish    Thomas Lennon  Comedy  20000000   139000000   
3          Ruby Dee    Quinton Aaron   Drama   1000000     2000000   
4       Cole Hauser       Mo Gallini  Action  76000000   236000000   

   Actors Box Office %  Director Box Office %   Earnings  \
0                50.00                  69.23   19400000   
1                

# 3. Mengecek Nilai yang Hilang

In [8]:
print("\n=== Cek Missing Values ===")
print(df.isnull().sum())


=== Cek Missing Values ===
Movie                                  0
Director                               0
Running time                           0
Actor 1                                0
Actor 2                                0
Actor 3                                2
Genre                                  0
Budget                                 0
Box Office                             0
Actors Box Office %                    0
Director Box Office %                  0
Earnings                               0
Oscar and Golden Globes nominations    0
Oscar and Golden Globes awards         3
Release year                           0
IMDb score                             0
dtype: int64


# 4. Perbaikan Nilai yang Hilang

In [10]:
# Kolom 'Actor 3' → isi dengan 'Unknown'
df_dirty['Actor 3'].fillna("Unknown", inplace=True)

# Kolom 'Oscar and Golden Globes awards' → isi dengan 'Unknown'
df_dirty['Oscar and Golden Globes awards'].fillna("Unknown", inplace=True)

/tmp/ipython-input-3386032815.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dirty['Oscar and Golden Globes awards'].fillna("Unknown", inplace=True)
/tmp/ipython-input-3386032815.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_dirty['Oscar and Golden Globes awards'].fillna("Unknown", inplace=True)


# 5. Cek Ulang Perbaikan Mising Velue

In [11]:
print("\n=== Missing Values Setelah Perbaikan ===")
print(df_dirty.isnull().sum())


=== Missing Values Setelah Perbaikan ===
Movie                                  0
Director                               0
Running time                           0
Actor 1                                0
Actor 2                                0
Actor 3                                0
Genre                                  0
Budget                                 0
Box Office                             0
Actors Box Office %                    0
Director Box Office %                  0
Earnings                               0
Oscar and Golden Globes nominations    0
Oscar and Golden Globes awards         0
Release year                           0
IMDb score                             0
dtype: int64


# 6. Cek & Hapus Duplikasi

In [12]:
print("\n=== Cek Data Duplikat ===")
dup_count = df_dirty.duplicated().sum()
print(f"Jumlah baris duplikat: {dup_count}")

before = df_dirty.shape[0]
df_clean = df_dirty.drop_duplicates()
after = df_clean.shape[0]


=== Cek Data Duplikat ===
Jumlah baris duplikat: 24


In [13]:
print(f"Jumlah baris sebelum cleaning: {before}")
print(f"Jumlah baris setelah cleaning: {after}")
print(f"Jumlah baris terhapus (duplikat): {before - after}")

Jumlah baris sebelum cleaning: 3974
Jumlah baris setelah cleaning: 3950
Jumlah baris terhapus (duplikat): 24


# 8. Standarisasi Kolom Box Office, Budget dan Earning

In [14]:
# Misalnya dataset kamu namanya df_dirty
comparison = df_dirty.copy()

# Kolom yang mau dikonversi jadi Million
cols_to_convert = ["Budget", "Box Office", "Earnings"]

# Pastikan numeric
for col in cols_to_convert:
    comparison[col] = pd.to_numeric(comparison[col], errors="coerce")

# Buat kolom Million di samping kolom aslinya
for col in cols_to_convert:
    new_col_name = col + "_Million"
    if new_col_name not in comparison.columns:
        comparison.insert(
            comparison.columns.get_loc(col) + 1,
            new_col_name,
            comparison[col].apply(lambda x: f"{x/1_000_000:.2f}M" if pd.notna(x) else "-")
        )

# Tampilkan 5 baris pertama
print("Dataset with added columns (Million) next to original:")
print(comparison.head(5))

# Simpan ke Google Drive
output_path = "/content/drive/MyDrive/Colab Notebooks/movies.xlsx"
comparison.to_excel(output_path, index=False)

print("\nDataset bersih disimpan di:", output_path)

Dataset with added columns (Million) next to original:
              Movie        Director  Running time          Actor 1  \
0          13 Hours     Michael Bay           144    Toby Stephens   
1         16 Blocks  Richard Donner           102     Bruce Willis   
2          17 Again     Burr Steers           102    Matthew Perry   
3              1982    Tommy Oliver            90  Bokeem Woodbine   
4  2 Fast 2 Furious  John Singleton           107      Paul Walker   

            Actor 2          Actor 3   Genre    Budget Budget_Million  \
0  James Badge Dale  David Costabile  Action  50000000         50.00M   
1       David Zayas       Sasha Roiz  Action  52000000         52.00M   
2    Hunter Parrish    Thomas Lennon  Comedy  20000000         20.00M   
3          Ruby Dee    Quinton Aaron   Drama   1000000          1.00M   
4       Cole Hauser       Mo Gallini  Action  76000000         76.00M   

   Box Office Box Office_Million  Actors Box Office %  Director Box Office %  \
0    

# 9. DATA ENRICHMENT

In [19]:
# Pastikan semua kolom numerik bertipe float
for col in df.select_dtypes(include=["number"]).columns:
    df[col] = df[col].astype(float)

changed_rows = []

# Loop tiap kolom numerik
for col in df.select_dtypes(include=["number"]).columns:
    if (df[col] == 0).any():  # hanya kalau ada nilai 0
        mean_val = df.loc[df[col] > 0, col].mean()

        # Simpan data sebelum diganti
        before = df.loc[df[col] == 0, [col]].copy()
        before = before.rename(columns={col: "Sebelum"})
        before["Kolom"] = col
        before["Sesudah"] = mean_val

        # Update dataframe (ganti 0 dengan mean)
        df.loc[df[col] == 0, col] = mean_val

        # Tambahkan ke list perubahan
        changed_rows.append(before)

# Gabungkan semua perubahan
if changed_rows:
    result = pd.concat(changed_rows, axis=0).reset_index(drop=True)

    # Atur tampilan angka supaya tidak pakai scientific notation
    pd.set_option("display.float_format", "{:,.0f}".format)

    print("=== Data yang berubah (sebelum → sesudah) ===")
    print(result.head(10))  # tampilkan 10 baris pertama
else:
    print("Tidak ada nilai 0 yang ditemukan.")

=== Data yang berubah (sebelum → sesudah) ===
   Sebelum                Kolom       Sesudah
0        0  Actors Box Office % 3,246,953,250
1        0  Actors Box Office % 3,246,953,250
2        0  Actors Box Office % 3,246,953,250
3        0  Actors Box Office % 3,246,953,250
4        0  Actors Box Office % 3,246,953,250
5        0  Actors Box Office % 3,246,953,250
6        0  Actors Box Office % 3,246,953,250
7        0  Actors Box Office % 3,246,953,250
8        0  Actors Box Office % 3,246,953,250
9        0  Actors Box Office % 3,246,953,250


# 10. Mencek Kolom Tidak Relevan

In [20]:
# 1. Cek jumlah nilai unik per kolom
print("=== Jumlah nilai unik per kolom ===")
print(df.nunique())

=== Jumlah nilai unik per kolom ===
Movie                                  3907
Director                               1760
Running time                            155
Actor 1                                1591
Actor 2                                2367
Actor 3                                2781
Genre                                    14
Budget                                  374
Box Office                              994
Actors Box Office %                     234
Director Box Office %                    39
Earnings                               1244
Oscar and Golden Globes nominations      22
Oscar and Golden Globes awards           14
Release year                             86
IMDb score                               77
dtype: int64


In [21]:
# Identifikasi kolom kandidat tidak relevan
irrelevant_cols = []

# Kolom dengan 1 nilai unik
single_value_cols = df.nunique()[df.nunique() == 1].index.tolist()
irrelevant_cols.extend(single_value_cols)

In [22]:
# Hapus kolom tidak relevan
df = df.drop(columns=set(irrelevant_cols), errors='ignore')

print("\n=== Setelah perbaikan ===")
print("Kolom dataset sekarang:", df.columns.tolist())


=== Setelah perbaikan ===
Kolom dataset sekarang: ['Movie', 'Director', 'Running time', 'Actor 1', 'Actor 2', 'Actor 3', 'Genre', 'Budget', 'Box Office', 'Actors Box Office %', 'Director Box Office %', 'Earnings', 'Oscar and Golden Globes nominations', 'Oscar and Golden Globes awards', 'Release year', 'IMDb score']


# 11. Menampilkan Data Yang Sudah Di Cleansing

In [23]:
print("\nContoh data bersih:")
display(df.head(10))


Contoh data bersih:


,Movie,Director,Running time,Actor 1,Actor 2,Actor 3,Genre,Budget,Box Office,Actors Box Office %,Director Box Office %,Earnings,Oscar and Golden Globes nominations,Oscar and Golden Globes awards,Release year,IMDb score
0,13 Hours,Michael Bay,144,Toby Stephens,James Badge Dale,David Costabile,Action,"50,000,000","69,400,000",50,69,"19,400,000",4,3,"2,016",7
1,16 Blocks,Richard Donner,102,Bruce Willis,David Zayas,Sasha Roiz,Action,"52,000,000","65,000,000",33,50,"13,000,000",4,3,"2,006",7
2,17 Again,Burr Steers,102,Matthew Perry,Hunter Parrish,Thomas Lennon,Comedy,"20,000,000","139,000,000",44,25,"119,000,000",4,3,"2,009",6
3,1982,Tommy Oliver,90,Bokeem Woodbine,Ruby Dee,Quinton Aaron,Drama,"1,000,000","2,000,000",50,100,"1,000,000",4,3,"2,013",7
4,2 Fast 2 Furious,John Singleton,107,Paul Walker,Cole Hauser,Mo Gallini,Action,"76,000,000","236,000,000",75,43,"160,000,000",4,3,"2,003",6
5,2 Guns,Baltasar Kormákur,109,Denzel Washington,Patrick Fischler,Fred Ward,Action,"61,000,000","132,000,000",67,100,"71,000,000",4,3,"2,013",7
6,2012,Roland Emmerich,158,Oliver Platt,Liam James,Tom McCarthy,Action,"200,000,000","800,000,000",64,50,"600,000,000",4,3,"2,009",6
7,21,Robert Luketic,123,Kevin Spacey,Jim Sturgess,Josh Gad,Crime,"35,000,000","81,000,000",61,67,"46,000,000",4,3,"2,008",7
8,22 Jump Street,Phil Lord,112,Channing Tatum,Craig Roberts,Amber Stevens West,Action,"50,000,000","331,000,000",56,100,"281,000,000",4,3,"2,014",7
9,27 Dresses,Anne Fletcher,111,Judy Greer,Bern Cohen,Yetta Gottesman,Comedy,"30,000,000","160,000,000",77,60,"130,000,000",4,3,"2,008",6
